<a href="https://colab.research.google.com/github/lingyu001/Recommendation/blob/master/YoutubeDNN_replicate.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

In [1]:
! wget http://files.grouplens.org/datasets/movielens/ml-1m.zip -O ./ml-1m.zip 
! wget https://raw.githubusercontent.com/shenweichen/DeepMatch/master/examples/preprocess.py -O preprocess.py
! unzip -o ml-1m.zip 
! pip install -q deepmatch

--2022-09-14 18:30:55--  http://files.grouplens.org/datasets/movielens/ml-1m.zip
Resolving files.grouplens.org (files.grouplens.org)... 128.101.65.152
Connecting to files.grouplens.org (files.grouplens.org)|128.101.65.152|:80... connected.
HTTP request sent, awaiting response... 200 OK
Length: 5917549 (5.6M) [application/zip]
Saving to: ‘./ml-1m.zip’

./ml-1m.zip         100%[===================>]   5.64M  27.7MB/s    in 0.2s    

2022-09-14 18:30:55 (27.7 MB/s) - ‘./ml-1m.zip’ saved [5917549/5917549]

--2022-09-14 18:30:55--  https://raw.githubusercontent.com/shenweichen/DeepMatch/master/examples/preprocess.py
Resolving raw.githubusercontent.com (raw.githubusercontent.com)... 185.199.108.133, 185.199.109.133, 185.199.110.133, ...
Connecting to raw.githubusercontent.com (raw.githubusercontent.com)|185.199.108.133|:443... connected.
HTTP request sent, awaiting response... 200 OK
Length: 6705 (6.5K) [text/plain]
Saving to: ‘preprocess.py’

preprocess.py       100%[===================>]  

In [33]:
import pandas as pd
from deepctr.feature_column import SparseFeat, VarLenSparseFeat
from preprocess import gen_data_set, gen_model_input
from sklearn.preprocessing import LabelEncoder
from tensorflow.python.keras import backend as K
from tensorflow.python.keras.models import Model

from deepmatch.models import YoutubeDNN
from deepmatch.utils import sampledsoftmaxloss, NegativeSampler

# Data 

In [4]:
data_path = "./"

unames = ['user_id','gender','age','occupation','zip']
user = pd.read_csv(data_path+'ml-1m/users.dat',sep='::',header=None,names=unames)
rnames = ['user_id','movie_id','rating','timestamp']
ratings = pd.read_csv(data_path+'ml-1m/ratings.dat',sep='::',header=None,names=rnames)
mnames = ['movie_id','title','genres']
movies = pd.read_csv(data_path+'ml-1m/movies.dat',sep='::',header=None,names=mnames,encoding="unicode_escape")
movies['genres'] = list(map(lambda x: x.split('|')[0], movies['genres'].values))

data = pd.merge(pd.merge(ratings,movies),user).iloc[:10000] # taking 1w

/usr/local/lib/python3.7/dist-packages/pandas/util/_decorators.py:311: ParserWarning: Falling back to the 'python' engine because the 'c' engine does not support regex separators (separators > 1 char and different from '\s+' are interpreted as regex); you can avoid this warning by specifying engine='python'.
  return func(*args, **kwargs)


In [55]:
data.user_id.nunique(), data.movie_id.nunique(), data.head(2)

(46, 2175,       user_id  movie_id  rating  timestamp  \
 9995       46       644       5  977089623   
 9998       46      1709       4  977089820   
 
                                        title  genres  gender  age  occupation  \
 9995  One Flew Over the Cuckoo's Nest (1975)       8       2    5          11   
 9998                     Verdict, The (1982)       8       2    5          11   
 
       zip  
 9995    2  
 9998    2  )

# Feature Engineering

In [6]:
sparse_features = ["movie_id", "user_id",
                    "gender", "age", "occupation", "zip", "genres"]

SEQ_LEN = 50
negsample = 0


In [7]:
feature_max_idx = {}
for feature in sparse_features:
    lbe = LabelEncoder()
    data[feature] = lbe.fit_transform(data[feature]) + 1
    feature_max_idx[feature] = data[feature].max() + 1

In [8]:
data.head()

,user_id,movie_id,rating,timestamp,title,genres,gender,age,occupation,zip
0,1,644,5,978300760,One Flew Over the Cuckoo's Nest (1975),8,1,1,9,18
1,1,385,3,978302109,James and the Giant Peach (1996),3,1,1,9,18
2,1,483,3,978301968,My Fair Lady (1964),12,1,1,9,18
3,1,1881,4,978300275,Erin Brockovich (2000),8,1,1,9,18
4,1,1306,5,978824291,"Bug's Life, A (1998)",3,1,1,9,18


In [9]:
user_profile = data[["user_id", "gender", "age", "occupation", "zip"]].drop_duplicates('user_id')
user_profile.set_index("user_id", inplace=True)
item_profile = data[["movie_id"]].drop_duplicates('movie_id')

In [10]:
user_item_list = data.groupby("user_id")['movie_id'].apply(list)

In [11]:
train_set, test_set = gen_data_set(data, SEQ_LEN, negsample)

100%|██████████| 46/46 [00:00<00:00, 966.13it/s]

8 8


In [12]:
data.loc[2]

user_id                         1
movie_id                      483
rating                          3
timestamp               978301968
title         My Fair Lady (1964)
genres                         12
gender                          1
age                             1
occupation                      9
zip                            18
Name: 2, dtype: object

In [41]:
print(train_set[0])

# reviewerID,  - user id
# pos_list[i], - movie id
# 1, - constant
# hist[::-1][:seq_len], - most recent watched movie id list before watching this movie, up to 50
# seq_len,  - 50
# genres_hist[::-1][:seq_len], - most recent watched movie's genres list before watching this movie, up to 50
# genres_list[i], - this movie's genre
# rating_list[i] - this movie's rating

(27, 939, 1, [355, 215, 721, 86, 1404, 15, 1342, 1671, 1545, 1069, 565, 325, 679, 1114, 715, 1956, 1076, 71, 1, 276, 1438, 1148, 558, 30, 487, 1735, 1599, 1742, 704, 1255, 1011, 1890, 365, 1285, 174, 1121, 646, 799, 688, 152, 656, 897, 645, 357, 674, 662, 187, 320, 1570, 953], 50, [2, 5, 1, 8, 8, 8, 5, 5, 16, 1, 1, 8, 8, 8, 5, 5, 8, 1, 3, 1, 1, 3, 4, 8, 2, 8, 2, 3, 2, 6, 8, 5, 6, 5, 6, 1, 1, 5, 1, 1, 8, 6, 1, 8, 8, 6, 8, 8, 5, 8], 8, 2)


In [47]:
train_model_input, train_label = gen_model_input(train_set, user_profile, SEQ_LEN)
test_model_input, test_label = gen_model_input(test_set, user_profile, SEQ_LEN)
train_label.shape, train_model_input['user_id'].shape, test_model_input['user_id'].shape, test_label.shape

/content/preprocess.py:86: VisibleDeprecationWarning: Creating an ndarray from ragged nested sequences (which is a list-or-tuple of lists-or-tuples-or ndarrays with different lengths or shapes) is deprecated. If you meant to do this, you must specify 'dtype=object' when creating the ndarray.
  train_seq_genres = np.array([line[5] for line in train_set])


((9908,), (9908,), (46,), (46,))

In [59]:
train_model_input

{'user_id': array([27,  5, 40, ...,  4, 19, 10]),
 'movie_id': array([ 939,  176, 1399, ..., 1545, 1087, 1859]),
 'hist_movie_id': array([[ 355,  215,  721, ...,  320, 1570,  953],
        [1404,  710,  255, ..., 1931, 1911, 2137],
        [ 339, 1812,  306, ..., 1772, 1495, 1591],
        ...,
        [1449, 1340, 2083, ...,    0,    0,    0],
        [ 584,  137, 1619, ...,  939, 1921,  757],
        [1662, 2126, 1941, ...,  485,  693,  604]], dtype=int32),
 'hist_genres': array([[ 2,  5,  1, ...,  8,  5,  8],
        [ 8,  5,  8, ..., 16,  8, 15],
        [ 1,  5,  8, ...,  8,  5,  5],
        ...,
        [ 5,  3,  1, ...,  0,  0,  0],
        [ 2,  5,  5, ...,  8,  8, 11],
        [16,  1, 16, ...,  5,  8,  4]], dtype=int32),
 'hist_len': array([50, 50, 50, ..., 40, 50, 50]),
 'genres': array([ 8,  8,  5, ..., 16, 11,  5]),
 'gender': array([1, 2, 1, ..., 2, 2, 2]),
 'age': array([3, 6, 3, ..., 3, 3, 5]),
 'occupation': array([13,  2, 17, ...,  6,  3,  4]),
 'zip': array([32, 40, 

# Aggregate Feature and create Embeddings

In [50]:
embedding_dim = 32

In [51]:
user_feature_columns = [SparseFeat('user_id', feature_max_idx['user_id'], 16),
                        SparseFeat("gender", feature_max_idx['gender'], 16),
                        SparseFeat("age", feature_max_idx['age'], 16),
                        SparseFeat("occupation", feature_max_idx['occupation'], 16),
                        SparseFeat("zip", feature_max_idx['zip'], 16),
                        VarLenSparseFeat(SparseFeat('hist_movie_id', feature_max_idx['movie_id'], embedding_dim,
                                                    embedding_name="movie_id"), SEQ_LEN, 'mean', 'hist_len'),
                        VarLenSparseFeat(SparseFeat('hist_genres', feature_max_idx['genres'], embedding_dim,
                                                   embedding_name="genres"), SEQ_LEN, 'mean', 'hist_len'),
                        ]

In [53]:
item_feature_columns = [SparseFeat('movie_id', feature_max_idx['movie_id'], embedding_dim)]


# Negative Sampling

In [67]:
from collections import Counter
train_counter = Counter(train_model_input['movie_id'])
item_count = [train_counter.get(i,0) for i in range(item_feature_columns[0].vocabulary_size)]
sampler_config = NegativeSampler('frequency',num_sampled=255,item_name="movie_id",item_count=item_count)

In [70]:
sampler_config

NegativeSampler(sampler='frequency', num_sampled=255, item_name='movie_id', item_count=[0, 20, 8, 3, 1, 3, 7, 3, 1, 9, 9, 1, 2, 1, 6, 13, 2, 4, 1, 10, 3, 8, 10, 1, 2, 6, 2, 17, 19, 2, 11, 8, 2, 1, 1, 6, 15, 8, 17, 2, 1, 1, 4, 1, 3, 1, 4, 1, 1, 4, 1, 1, 3, 1, 3, 2, 2, 4, 1, 3, 1, 2, 2, 2, 5, 1, 3, 3, 6, 2, 3, 22, 17, 3, 1, 1, 1, 1, 1, 1, 1, 8, 5, 1, 1, 1, 9, 4, 7, 1, 1, 2, 2, 2, 5, 4, 6, 7, 4, 6, 3, 3, 2, 5, 5, 4, 1, 1, 1, 4, 3, 1, 5, 1, 4, 3, 2, 4, 6, 3, 2, 2, 1, 1, 1, 4, 1, 1, 4, 17, 6, 2, 3, 7, 3, 2, 1, 12, 4, 2, 1, 2, 2, 5, 5, 1, 4, 1, 1, 3, 1, 2, 30, 4, 1, 11, 7, 1, 1, 3, 3, 1, 1, 2, 2, 3, 2, 2, 9, 1, 3, 1, 4, 8, 25, 2, 8, 4, 3, 1, 1, 1, 1, 1, 2, 11, 4, 31, 5, 1, 2, 1, 1, 5, 7, 3, 3, 1, 1, 9, 1, 4, 2, 1, 4, 4, 6, 2, 2, 10, 3, 1, 1, 8, 2, 19, 12, 1, 2, 1, 20, 1, 12, 11, 5, 1, 7, 1, 1, 2, 16, 4, 13, 2, 2, 3, 1, 1, 1, 2, 1, 6, 3, 2, 2, 1, 2, 1, 1, 4, 1, 1, 2, 1, 3, 2, 1, 4, 3, 5, 4, 2, 1, 5, 7, 6, 1, 4, 1, 1, 3, 2, 1, 9, 1, 1, 20, 1, 2, 2, 1, 2, 3, 6, 1, 8, 2, 27, 1, 2, 2, 1, 2, 4, 2,